# Hospital Readmission Prediction – End-to-End Healthcare Analytics

This notebook walks through a complete **Hospital Readmission Prediction** project: from business context to model evaluation and deployment considerations, using Python and Scikit-learn.

---
## 1. Business Problem Understanding

### Why hospital readmission matters
- **Quality of care**: Readmissions within 30 days often indicate inadequate discharge planning, follow-up, or patient understanding.
- **Patient outcomes**: Repeated hospitalizations increase risk of complications and reduce quality of life.
- **Regulatory**: In many countries (e.g., US HRRP), hospitals are penalized for excess readmissions.

### Cost impact
- Unplanned readmissions are expensive (e.g., $15–20K+ per readmission in the US).
- Medicare and other payers may reduce reimbursement for high readmission rates.
- Proactive interventions (care coordination, follow-up) are cheaper than readmissions.

### Healthcare operational efficiency
- Identifying high-risk patients allows targeted discharge planning and transitional care.
- Better bed utilization and scheduling when readmission risk is factored in.
- Data-driven prioritization of case management and follow-up calls.

---
## 2. Dataset Description

We use a synthetic dataset that mirrors real-world structure:

| Category | Variables | Description |
|----------|-----------|-------------|
| **Patient demographics** | age, gender, race | Age group, sex, race/ethnicity |
| **Medical history** | number_outpatient, number_emergency, number_inpatient | Prior utilization |
| **Admission details** | admission_type, admission_source, discharge_disposition, time_in_hospital, admission_date | Type of stay and length |
| **Lab / procedures** | num_lab_procedures, num_procedures, num_medications, max_glu_serum, A1Cresult | Labs and meds |
| **Diagnosis** | diag_1, diag_2, diag_3, number_diagnoses | Primary/secondary diagnoses |
| **Chronic conditions** | diabetes, hypertension, chronic_kidney_disease, heart_failure | Binary indicators |
| **Target** | readmitted | 1 = readmitted within 30 days, 0 = not readmitted |

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
DATA_PATH = PROJECT_ROOT / 'data' / 'hospital_readmission.csv'

if not DATA_PATH.exists():
    sys.path.insert(0, str(PROJECT_ROOT / 'src'))
    from generate_data import generate_synthetic_readmission_data
    (PROJECT_ROOT / 'data').mkdir(exist_ok=True)
    df_raw = generate_synthetic_readmission_data()
    df_raw.to_csv(DATA_PATH, index=False)
    print('Generated synthetic data at', DATA_PATH)

df = pd.read_csv(DATA_PATH)
print('Shape:', df.shape)
df.head(10)

In [ ]:
df.info()
df.describe(include='all').T

---
## 3. Data Cleaning

- Handle missing values
- Remove duplicates
- Encode categorical variables
- Outlier detection

In [ ]:
df_clean = df.copy()

missing = df_clean.isnull().sum()
missing_pct = (missing / len(df_clean) * 100).round(2)
pd.DataFrame({'missing_count': missing, 'missing_pct': missing_pct}).query('missing_count > 0')

In [ ]:
cat_cols = df_clean.select_dtypes(include=['object']).columns.tolist()
for c in cat_cols:
    if df_clean[c].isnull().any():
        df_clean[c] = df_clean[c].fillna(df_clean[c].mode().iloc[0] if df_clean[c].mode().notna().any() else 'Unknown')

if 'patient_id' in df_clean.columns:
    df_clean = df_clean.drop_duplicates(subset=['patient_id'], keep='first')
else:
    df_clean = df_clean.drop_duplicates()
print('After dropping duplicates:', df_clean.shape)

In [ ]:
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in ['patient_id', 'readmitted']]

def count_outliers_iqr(series):
    Q1, Q3 = series.quantile(0.25), series.quantile(0.75)
    IQR = Q3 - Q1
    low, high = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    return ((series < low) | (series > high)).sum()

outliers = {c: count_outliers_iqr(df_clean[c]) for c in numeric_cols}
pd.Series(outliers).sort_values(ascending=False)

In [ ]:
cap_cols = ['time_in_hospital', 'num_lab_procedures', 'num_medications', 'number_diagnoses']
for c in cap_cols:
    if c in df_clean.columns:
        q99 = df_clean[c].quantile(0.99)
        df_clean[c] = df_clean[c].clip(upper=q99)
print('Outliers capped at 99th percentile for:', cap_cols)

In [ ]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df_clean.copy()
label_encoders = {}
for c in cat_cols:
    le = LabelEncoder()
    df_encoded[c] = le.fit_transform(df_encoded[c].astype(str))
    label_encoders[c] = le
print('Encoded columns:', list(label_encoders.keys()))
df_encoded.head()

---
## 4. Exploratory Data Analysis

- Readmission rate
- Age vs readmission
- Diagnosis vs readmission
- Length of stay impact
- Correlation heatmap

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

ax = axes[0, 0]
readm_rate = df_clean['readmitted'].mean()
ax.bar(['Not readmitted', 'Readmitted'], [1 - readm_rate, readm_rate], color=['steelblue', 'coral'])
ax.set_ylabel('Proportion')
ax.set_title(f'Readmission rate = {readm_rate:.2%}')

ax = axes[0, 1]
age_readm = df_clean.groupby('age')['readmitted'].mean().sort_index()
age_readm.plot(kind='bar', ax=ax, color='teal', alpha=0.8)
ax.set_title('Readmission rate by age group')
ax.set_ylabel('Readmission rate')
ax.tick_params(axis='x', rotation=45)

ax = axes[1, 0]
diag_readm = df_clean.groupby('diag_1')['readmitted'].mean().sort_values(ascending=False)
diag_readm.plot(kind='barh', ax=ax, color='mediumpurple', alpha=0.8)
ax.set_title('Readmission rate by primary diagnosis')
ax.set_xlabel('Readmission rate')

ax = axes[1, 1]
los_bins = pd.cut(df_clean['time_in_hospital'], bins=[0, 3, 5, 7, 14], labels=['1-3', '4-5', '6-7', '8+'])
los_readm = df_clean.assign(los_bin=los_bins).groupby('los_bin')['readmitted'].mean()
los_readm.plot(kind='bar', ax=ax, color='darkorange', alpha=0.8)
ax.set_title('Readmission rate by length of stay (days)')
ax.set_ylabel('Readmission rate')
ax.set_xlabel('Length of stay')

plt.tight_layout()
plt.show()

In [ ]:
num_for_corr = df_encoded.select_dtypes(include=[np.number]).columns.tolist()
corr = df_encoded[num_for_corr].corr()
plt.figure(figsize=(14, 10))
sns.heatmap(corr, annot=False, cmap='RdBu_r', center=0, vmin=-0.5, vmax=0.5)
plt.title('Correlation heatmap (encoded + numeric features)')
plt.tight_layout()
plt.show()

---
## 5. Feature Engineering

- Create risk score features
- Convert admission dates
- Binning age groups
- Chronic condition indicators

In [ ]:
df_fe = df_clean.copy()

if 'admission_date' in df_fe.columns:
    df_fe['admission_date'] = pd.to_datetime(df_fe['admission_date'], errors='coerce')
    df_fe['admission_month'] = df_fe['admission_date'].dt.month
    df_fe['admission_dayofweek'] = df_fe['admission_date'].dt.dayofweek
    df_fe['admission_weekend'] = (df_fe['admission_dayofweek'] >= 5).astype(int)

age_order = ['[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)', '[60-70)', '[70-80)', '[80-90)', '[90-100)']
df_fe['age_group_idx'] = df_fe['age'].map(lambda x: age_order.index(x) if x in age_order else -1).clip(0, 9)

chronic_cols = [c for c in ['diabetes', 'hypertension', 'chronic_kidney_disease', 'heart_failure'] if c in df_fe.columns]
df_fe['chronic_count'] = df_fe[chronic_cols].sum(axis=1)

df_fe['risk_heuristic'] = (
    (df_fe['time_in_hospital'] >= 7).astype(int) * 2 +
    (df_fe['num_medications'] >= 15).astype(int) * 1 +
    (df_fe['number_diagnoses'] >= 8).astype(int) * 1 +
    df_fe['chronic_count']
)

print('New features:', [c for c in df_fe.columns if c not in df_clean.columns])

In [ ]:
cat_cols_fe = df_fe.select_dtypes(include=['object']).columns.tolist()
df_model = df_fe.copy()
for c in cat_cols_fe:
    df_model[c] = LabelEncoder().fit_transform(df_model[c].astype(str))

feature_cols = [c for c in df_model.columns if c not in ['patient_id', 'readmitted', 'admission_date']]
X = df_model[feature_cols]
y = df_model['readmitted']
print('Feature matrix shape:', X.shape)
print('Features:', feature_cols[:15], '...' if len(feature_cols) > 15 else '')

---
## 6. Model Building

- Train-test split
- Cross-validation
- Logistic Regression, Random Forest, XGBoost

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=12, random_state=42, class_weight='balanced'),
}

print('Cross-validation (F1 weighted):')
for name, model in models.items():
    if name == 'Logistic Regression':
        scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='f1_weighted')
    else:
        scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='f1_weighted')
    print(f'  {name}: {scores.mean():.4f} (+/- {scores.std()*2:.4f})')

In [ ]:
try:
    import xgboost as xgb
    xgb_model = xgb.XGBClassifier(n_estimators=100, max_depth=6, random_state=42, use_label_encoder=False, eval_metric='logloss')
    scores = cross_val_score(xgb_model, X_train, y_train, cv=cv, scoring='f1_weighted')
    print(f'  XGBoost: {scores.mean():.4f} (+/- {scores.std()*2:.4f})')
    models['XGBoost'] = xgb_model
except ImportError:
    print('XGBoost not installed; skipping.')

In [ ]:
lr = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
lr.fit(X_train_scaled, y_train)

rf = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)

if 'XGBoost' in models:
    models['XGBoost'].fit(X_train, y_train)
print('Models fitted.')

---
## 7. Model Evaluation

Accuracy, Precision, Recall, F1, ROC-AUC, Confusion Matrix

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

def evaluate_model(name, y_true, y_pred, y_proba=None):
    return {
        'Model': name,
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1': f1_score(y_true, y_pred, zero_division=0),
        'ROC-AUC': roc_auc_score(y_true, y_proba[:, 1]) if y_proba is not None else None
    }

results = []

y_pred_lr = lr.predict(X_test_scaled)
y_proba_lr = lr.predict_proba(X_test_scaled)
results.append(evaluate_model('Logistic Regression', y_test, y_pred_lr, y_proba_lr))

y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)
results.append(evaluate_model('Random Forest', y_test, y_pred_rf, y_proba_rf))

if 'XGBoost' in models:
    y_pred_xgb = models['XGBoost'].predict(X_test)
    y_proba_xgb = models['XGBoost'].predict_proba(X_test)
    results.append(evaluate_model('XGBoost', y_test, y_pred_xgb, y_proba_xgb))

metrics_df = pd.DataFrame(results)
metrics_df

In [ ]:
print('Classification report (Random Forest):')
print(classification_report(y_test, y_pred_rf, target_names=['Not readmitted', 'Readmitted']))

fig, ax = plt.subplots(1, 1, figsize=(5, 4))
cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Not readmitted', 'Readmitted'],
            yticklabels=['Not readmitted', 'Readmitted'])
ax.set_ylabel('True')
ax.set_xlabel('Predicted')
ax.set_title('Confusion Matrix (Random Forest)')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import RocCurveDisplay

fig, ax = plt.subplots(1, 1, figsize=(6, 5))
RocCurveDisplay.from_predictions(y_test, y_proba_lr[:, 1], name='Logistic Regression', ax=ax)
RocCurveDisplay.from_predictions(y_test, y_proba_rf[:, 1], name='Random Forest', ax=ax)
if 'XGBoost' in models:
    RocCurveDisplay.from_predictions(y_test, y_proba_xgb[:, 1], name='XGBoost', ax=ax)
ax.plot([0, 1], [0, 1], 'k--')
ax.set_title('ROC curves')
plt.tight_layout()
plt.show()

---
## 8. Feature Importance Analysis

In [ ]:
imp = pd.DataFrame({'feature': feature_cols, 'importance': rf.feature_importances_})
imp = imp.sort_values('importance', ascending=False)
imp.head(15)

In [ ]:
plt.figure(figsize=(10, 8))
top_n = 20
imp_top = imp.head(top_n)
plt.barh(range(len(imp_top)), imp_top['importance'].values, color='steelblue', alpha=0.8)
plt.yticks(range(len(imp_top)), imp_top['feature'].values)
plt.gca().invert_yaxis()
plt.xlabel('Feature importance (Random Forest)')
plt.title('Top 20 features')
plt.tight_layout()
plt.show()

---
## 9. Business Interpretation

- **High-risk patient identification**: Use model probability (e.g., >0.3 or top decile) to flag patients for enhanced discharge planning and follow-up.
- **Risk segmentation**: Segment into Low / Medium / High risk based on predicted probability thresholds; allocate case management resources accordingly.
- **Operational recommendations**:
  1. Integrate risk score into discharge workflow (checklist, follow-up scheduling).
  2. Prioritize post-discharge calls and home visits for high-risk patients.
  3. Focus on modifiable drivers (medication reconciliation, primary care follow-up) highlighted by feature importance.
  4. Monitor readmission rates by segment to validate model and adjust thresholds.

In [ ]:
prob_rf = y_proba_rf[:, 1]
df_test = pd.DataFrame({'y_true': y_test.values, 'prob': prob_rf})
df_test['segment'] = pd.cut(prob_rf, bins=[0, 0.2, 0.4, 1.0], labels=['Low', 'Medium', 'High'])
segment_summary = df_test.groupby('segment').agg(
    count=('prob', 'count'),
    actual_readmission_rate=('y_true', 'mean')
).round(4)
segment_summary

---
## 10. Deployment Consideration

- **How the hospital can use this model**:
  - **Real-time**: At discharge, run the model on current encounter features and attach a risk score to the record; trigger alerts for high-risk patients.
  - **Batch**: Daily or weekly batch scoring of recent discharges to generate lists for care managers.
  - **Dashboard**: Show readmission risk distribution and trends; drill-down by unit, diagnosis, or physician.

- **Monitoring strategy**:
  1. **Performance**: Track precision/recall and ROC-AUC on a holdout or recent period; set alerts for drift (e.g., AUC drop > 0.05).
  2. **Data drift**: Monitor distributions of key inputs (e.g., time_in_hospital, num_medications) and retrain if they shift substantially.
  3. **Outcomes**: Compare actual 30-day readmission rates by risk segment to validate that high-risk segment has higher observed rate.
  4. **Fairness**: Stratify metrics by age, race, and payer to ensure no unintended bias; recalibrate or constrain model if needed.

In [ ]:
print('Project complete. Summary:')
print(f'  - Dataset: {len(df)} rows, readmission rate {y.mean():.2%}')
print(f'  - Best F1 (from metrics): {metrics_df["F1"].max():.4f}')
print('  - Next steps: Export model (e.g. joblib), build API or batch job, set up monitoring.')